<img width="8%" alt="GitHub.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/GitHub.png" style="border-radius: 15%">

# GitHub - Remove directories with branches closed on my local
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=GitHub+-+Remove+directories+with+branches+closed+on+my+local:+Error+short+description">Bug report</a>

**Tags:** #github #branches #list #api #rest #python #active

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2023-07-24 (Created: 2023-07-24)

**Description:** This notebook facilitates the removal of directories associated with branches on your local machine. If you need to clone and create directories based on active branches, you can use either of the following notebooks: GitHub_Clone_open_branches_from_repository_on_my_local or GitHub_Clone_repository_and_switch_branch.

**References:**
- [GitHub - List branches](https://docs.github.com/en/rest/branches/branches?apiVersion=2022-11-28#list-branches)
- [GitHub - Authentication](https://docs.github.com/en/rest/overview/resources-in-the-rest-api#authentication)

## Input

### Import libraries

In [ ]:
import naas
import pandas as pd
import requests
import shutil
import os
from datetime import datetime, timedelta

### Setup Variables
- `token`: [Generate a personal access token](https://docs.github.com/en/github/authenticating-to-github/creating-a-personal-access-token)
- `repo_url`: URL of the repository to clone
- `days_limit`: The number of days a branch can remain on your local after the associated issue has been closed.
- `output_dir`: Output directory to clone repo. If None, we will create a folder with the name of the repo

In [ ]:
# Inputs
token = naas.secret.get(name="GITHUB_TOKEN") or "YOUR_GITHUB_TOKEN"
repo_url = "https://github.com/jupyter-naas/awesome-notebooks"
days_limit = 30

# Outputs
output_dir = None

## Model

### List local branches

In [ ]:
output_branches = sorted(os.listdir(output_dir))
print(f"Branches on my local", len(output_branches))

### List active branches with open PR's

In [ ]:
def get_branches_with_open_prs(
    token,
    repo_url
):
    # Init
    data = []
    owner = repo_url.split("https://github.com/")[-1].split("/")[0]
    repository = repo_url.split(f"https://github.com/{owner}/")[-1].split("/")[0]
    
    # Requests
    url = f"https://api.github.com/repos/{owner}/{repository}/pulls"
    headers = {"Authorization": f"token {token}"}
    response = requests.get(url, headers=headers)
    pulls = response.json()
    
    # Data
    for pull in pulls:
        branch = pull['head']['ref']
        creator = pull['user']['login']
        creation_date = pull['created_at']
        
        data.append({
            'branch': branch,
            'creator': creator,
            'creation_date': creation_date
        })
    
    df = pd.DataFrame(data)
    
    # Sort values
    if len(df) > 0:
        df = df.sort_values(by="creation_date", ascending=False)
    return df.reset_index(drop=True)

branches_with_open_prs = get_branches_with_open_prs(token, repo_url)
print(f"Branches with open pull requests in {repo_url}:", len(branches_with_open_prs))
branches_with_open_prs.head(1)

## Output

### Removed branches on my local

In [ ]:
active_branches = branches_with_open_prs["branch"].unique()

# Loop
for branch in output_branches:
    if not output_dir:
        output_dir = '/home/ftp/'
    folder = os.path.join(output_dir, branch)
    
    # Extract the relevant information from the metadata
    folder_stat = os.stat(folder).st_mtime
    
    # Convert timestamp to a datetime object
    datetime_obj = datetime.fromtimestamp(folder_stat)
    
    # Convert datetime object to a formatted date string
    date_string = datetime_obj.strftime('%Y-%m-%d %H:%M:%S')
    
    # Get the current date
    current_date = datetime.now()

    # Create limit date
    date_limit = current_date - timedelta(days=days_limit)
    
    # Remove folder on conditions
    if branch not in active_branches and not branch.startswith(".") and datetime_obj < date_limit:
        shutil.rmtree(folder)
        print(f"✅ Branch '{branch}' removed from outputs")